In [1]:
from __future__ import print_function 

#%load_ext autoreload
#%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist,mnist_m
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.training import *
from experiments.SL_bound import *
from util.kl import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
#batch_size = 128
num_classes = 10
make_plots = False

delta=0.05 ## what would this be?   

#alphas=[0]
#length=10
#for i in range(length-1):
#    alphas.append((i+1)/length)



sigmas=[]
for i in range(2,9):  
    sigmas.append([3,i])
    if(i==8):
        break
    sigmas.append([1,i])

img_rows, img_cols = 32, 32
project_folder = "/cephyr/users/adambre/Alvis/"


### TODO

# # check output_dir, create it if not exists
#     if not os.path.isdir(output_dir):
#         os.makedirs(output_dir)

# Load data

In [2]:
import time
import os
import numpy as np
import pandas as pd

def binarize(y,x):
    ## take in one hot label encoding and make it into either 'label x' or 'not label x'
    ## x is in [0,5] as we have at least 6 overlapping labels in our chestxray data
    ## labeldict={"No Finding":0,"Cardiomegaly":1,"Edema":2,"Consolidation":3,"Atelectasis":4,"Effusion":5}
    y_new=[]
    mask=np.zeros(6)
    mask[x]=1
    for i in y:
        if np.dot(i,mask)==1:
            ## we have the label
            y_new.append([0,1])
        else:
            ## we do not have the label
            y_new.append([1,0])
    return np.array(y_new)
def load_xray(img_size,label):
    #data_path2="/cephyr/NOBACKUP/groups/snic2021-23-538/"
    
    data_path="/cephyr/users/adambre/Alvis/mnist_transfer/"
    x_chest=np.load(data_path+"chestxray14_"+str(img_size)+".npy",allow_pickle=True)
    y_chest=np.load(data_path+"chestxray14_"+str(img_size)+"_labels.npy",allow_pickle=True)
    
    x_chex=np.load(data_path+"chexpert_"+str(img_size)+".npy",allow_pickle=True)
    y_chex=np.load(data_path+"chexpert_"+str(img_size)+"_labels.npy",allow_pickle=True)
    
    ### Binarize labels
    
    y1=binarize(y_chest,2)
    y2=binarize(y_chex,2)
   
    
    ### do standard scaling
    x_chest = x_chest.astype('float32')
    sigma=np.std(x_chest)
    x_chest /=sigma

    x_chex = x_chex.astype('float32')
    sigma2=np.std(x_chex)
    x_chex /=sigma
    ## mean subtraction
    mu=np.mean(x_chest)
    x_chest -= mu

    mu2=np.mean(x_chex)
    x_chex -= mu
    
    print('mean, variance', mu, sigma)
    print("---------------Load ChestXray14----------------")
    print(x_chest.shape, y1.shape)
    print('mean, variance', mu2, sigma2)
    print("---------------Load CheXpert----------------")
    print(x_chex.shape, y2.shape)
    
    ## add them to each other
    X=np.append(x_chest, x_chex,axis=0)
    y=np.append(y1, y2,axis=0)
 
    ### 
    x_source, x_target, y_source , y_target = train_test_split(X,y,test_size=0.5,random_state=69105)

    return x_source, x_target, y_source , y_target

#x_source, x_target, y_source , y_target=load_xray(32,2)
#x_train, x_test, y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=69105)
# x_source, y_source, x_target, y_target=load_task(6)
# M=init_task_model(6,True)

# k=0
# for y in y_source:
#     if np.dot(y,[1,0])==0: ## not [1,0]
#         k+=1
# k2=0
# for y in y_target:
#     if np.dot(y,[1,0])==0:
#         k2+=1
# print(k/len(y_source))
# print(k2/len(y_target))
# ## choose loss function, optimiser etc. and train
# M.compile(loss=tf.keras.losses.categorical_crossentropy,
#            optimizer=tf.keras.optimizers.SGD(learning_rate=0.00003, momentum=0.95),
#                   metrics=['accuracy'])

# M.evaluate(x_target,y_target)

# fit_info = M.fit(x_source, y_source,
#                  validation_data=(x_target,y_target),
#        batch_size=128,
#        epochs=50,
#        verbose=1,
#                     )
# M.evaluate(x_target,y_target)
# #print(fit_info.history)
# L=len(fit_info.history["val_accuracy"])
# x=np.linspace(1,L,num=L)
# plt.plot(x,fit_info.history["val_accuracy"])
# #print(sum(y_new))

In [3]:
TASK=6


# Define training functions

In [4]:
def init_tf():
    ### making sure that we have the GPU to work on
    gpus = tf.config.experimental.list_physical_devices('GPU')
    #logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    if gpus:
      # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
      # I do not know why I have to do this but gpu does not work otherwise.
        try:
            tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
            print(e)

    
## custom callback to terminate training at some specific value of a metric
    
class stop_callback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='accuracy', value=0.001, verbose=0):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy')> self.value): # select the accuracy
            print("\n !!! training error threshold reached, no further training !!!")
            self.model.stop_training = True
            
class fast_checkpoints(tf.keras.callbacks.Callback):
    def __init__(self,checkpoint_path,save_freq):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.save_freq=save_freq
        self.filepath=checkpoint_path
        self.verbose=1
        self.save_best_only=False
        self.save_weights_only=True
    def on_train_batch_begin(self, batch, epoch, logs=None):
         if batch%self.save_freq==0:

            #print("\n Saved weights at the start of batch"+str(batch)+"\n")

            ## Create folder
            weight_path = self.filepath+"/1_"+str(batch)+".ckpt"
            os.makedirs(os.path.dirname(weight_path), exist_ok=True)
            
            self.model.save_weights(weight_path)
            
def train_posterior(alpha,x_train,y_train,prior_weights=None,x_test=[],y_test=[],save=True,epsilon=0.01,Task=2,Binary=False,batch_size=128):
        
        TASK=Task
        
        ### x_test should be the whole of S for early stopping purposes
    
        checkpoint_path = "posteriors/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))
        if Binary:
            checkpoint_path = "posteriors/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))
        
        
        # Create a callback that saves the model's weights every epoch
        checkpoint_freq=np.ceil(len(x_train)/batch_size)
        #print(checkpoint_freq)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            save_freq="epoch",#int(checkpoint_freq),### 547 = ceiling(70000/128) i.e training set for MNIST/MNIST-M,
            filepath=checkpoint_path+"/2_{epoch:0d}.ckpt", 
            verbose=1,
            save_best_only=False,
            save_weights_only=True,
                ## tune when to save as needed for plots
        )
        ## callback for first part
        fast_checkpoint_freq=np.ceil(len(x_train)/(batch_size*10))
        fast_cp_callback =fast_checkpoints(checkpoint_path,int(fast_checkpoint_freq))
        stopping_callback=stop_callback(monitor='val_acc',value=1-epsilon)
    
        M=init_task_model(TASK,Binary)

        
            
        ## choose loss function, optimiser etc. and train
        
        M.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                      metrics=['accuracy'],)
        
        
        ## Create the folder
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        ## remove previous weights
        import glob
        files=glob.glob(os.path.join(checkpoint_path+'/*'))
        
        
        for file in files:
            if file==(checkpoint_path+'/params.txt'):
                files.remove(checkpoint_path+'/params.txt')
            os.remove(file)
        
        ### load the prior weights
        if prior_weights is not None:
            M.set_weights(prior_weights)
        elif(alpha==0):
            ### save the rand. init as the prior
            if Binary:
                prior_path="priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))+"/prior.ckpt"
            else:
                prior_path="priors/"+"task"+str(TASK)+"/"+str(int(100*alpha))+"/prior.ckpt"
            
            ## Create the folder
            os.makedirs(os.path.dirname(prior_path), exist_ok=True)

            ## Save the weights
            M.save_weights(prior_path)
        else:
            if Binary:
                prior_path="priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))+"/prior.ckpt"
                M.load_weights(prior_path).expect_partial()
            else:
                
                M.load_weights(prior_path).expect_partial()
        
    
        if save:
            CALLBACK=[fast_cp_callback,stopping_callback]
        else:
            CALLBACK=[stopping_callback]
        ### train for one epoch with more checkpoints to be able to plot more there
        fit_info = M.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=1, 
           callbacks=CALLBACK,
           validation_data=(x_test, y_test),
           verbose=0,
                        )
        
        print("-"*40)
        print("Finished training first posterior epoch")
        if save:
            CALLBACK=[cp_callback,stopping_callback]
        else:
            CALLBACK=[stopping_callback]
            
        fit_info = M.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=2000, # we should have done early stopping before this completes
           callbacks=CALLBACK,
           validation_data=(x_test, y_test),
           verbose=0,
                        )
        print("-"*40)
        print("Finished training posterior")
        
         #### save the last posterior weights to disk
        epochs_trained=len(fit_info.history['loss'])
        #if save:
            ## Create the folder
            #os.makedirs(os.path.dirname(checkpoint_path+"/2_"+str(epochs_trained)), exist_ok=True)
            
            
            #M.save_weights(checkpoint_path+"/2_"+str(epochs_trained)) ###### check if we need this; TODO!!!!!!
            
        #### save textfile with parameters, i.e. alpha ,epochs trained and epsilon
        if Binary:
            with open('posteriors/'+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+'/params.txt', 'w') as f:
                f.write('\n'.join([str(alpha), str(epsilon), str(epochs_trained)]))     
            f.close()
        else:
            with open('posteriors/'+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+'/params.txt', 'w') as f:
                f.write('\n'.join([str(alpha), str(epsilon), str(epochs_trained)]))     
            f.close()
        W=M.get_weights()
        return W
    
def train_prior(alpha,total_epochs,x_train=[],y_train=[],x_target=[],y_target=[],save=True,Task=2,Binary=False,batch_size=128):
    TASK=Task
    
    if Binary:
        ## Create the folders
        os.makedirs(os.path.dirname("priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha)))+"/", exist_ok=True)
        checkpoint_path = "priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))
    else:
        os.makedirs(os.path.dirname("priors/"+"task"+str(TASK)+"/"+str(int(100*alpha)))+"/", exist_ok=True)
        checkpoint_path = "priors/"+"task"+str(TASK)+"/"+str(int(100*alpha))
    ## remove previous weights
    import glob
    files=glob.glob(os.path.join(checkpoint_path+'/*'))
    for file in files:
        os.remove(file)
    M=init_task_model(TASK,Binary)
    #sys.exit(-1)
    ### save checkpoints 10 times over the training of the prior
    l=len(x_train)
    checkpoint_freq=np.ceil(l/(batch_size*10))
    fast_cp_callback =fast_checkpoints(checkpoint_path,int(checkpoint_freq))
    if save:
            CALLBACK=[fast_cp_callback]
    else:
            CALLBACK=[]
            
    ## choose loss function, optimiser etc. and train
    M.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                      metrics=['accuracy'],)
    fit_info = M.fit(x_train, y_train,
           batch_size=batch_size,
           callbacks=CALLBACK,
           epochs=total_epochs,
           verbose=0,
                        )
    print("-"*40)
    print("Finished training prior")
    #### save the final prior weights to disk
    if save:
        os.makedirs(checkpoint_path, exist_ok=True)
        M.save_weights(checkpoint_path+"/prior.ckpt")
     
    
    list1=[]
    
    dirFiles = os.listdir(checkpoint_path) #list of directory files
    
    ## remove the ckpt.index and sort so that we get the epochs that are in the directory
    for files in dirFiles: #filter out all non weights
        if '.ckpt.index' in files:
            name = re.sub('\.ckpt.index$', '', files)
            if (name[0]=="1"):
                list1.append(name)
        
    list1.sort(key=lambda f: int(re.sub('\D', '', f)))
    list1.append("prior")    ## add the final weights which has no number
    
    Ws=list1
    weight_updates=[]
    for i in Ws:
        if i[0]=="1":
            if i[1]=="_":
                weight_updates.append(int(i[2:]))
    weight_updates.append(int(np.ceil(len(y_train)/batch_size)))
    
    error=[]
    target_error=[]
    for checkpoint in Ws:

        model=init_task_model(TASK,Binary)
        model.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                  metrics=['accuracy'],)
      
            
        model.load_weights(checkpoint_path+"/"+str(checkpoint)+".ckpt").expect_partial()
        target_error.append(1-model.evaluate(x_target,y_target,verbose=0)[1])
        error.append(1-model.evaluate(x_train,y_train,verbose=0)[1])
    print("-"*40)
    print("Finished computing prior sample and target errors")
    if save:
        results=pd.DataFrame({'Weightupdates': weight_updates,
            'Trainerror': error,
            'targeterror':target_error,
            })
       
        ## Create the folders
        os.makedirs(os.path.dirname(project_folder+"mnist_transfer/results/"+"task"+str(TASK)+"/Binary/"), exist_ok=True)
        if Binary:
            with open(project_folder+"mnist_transfer/results/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))+"_prior_results.pkl",'wb') as f:
                pickle.dump(results,f)
            f.close()
        else:
            with open(project_folder+"mnist_transfer/results/"+"task"+str(TASK)+"/"+str(int(100*alpha))+"_prior_results.pkl",'wb') as f:
                pickle.dump(results,f)
            f.close()
    return model.get_weights()
    


def plot_result_file(epsilon,alpha,sigma,Binary=False,Task=2):
    sigma_tmp=sigma
    sigma=sigma[0]*10**(-1*sigma[1])
    
    if Binary:
        result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        plt.title("Binary: "+r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
    else:
        result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
    results=pd.read_pickle(result_path+"_results.pkl")
    
    ### do the plots
    plt.plot(results["Weightupdates"],results["Bound"],'r*-')
    plt.plot(results["Weightupdates"],results["Trainerror"],'m^-')
    
    plt.xlabel("Weight updates")
    plt.ylabel("Error")
    
    plt.legend(["Bound","Empirical error"])
    plt.show()

def find_optimal_sigma(sigmas,epsilon, alpha,Binary=False,Task=2):
    #### to find the optimal sigma just do a search through all the results 
    #### and save the one for each parameter which has the minimal bound
    #### Do we do this per epoch or for some other value? The sigma which yields the lowest bound overall for some epoch?
    optimal=[0,1]
    # search through all epochs and pick the sigma which yields the smallest bound during the whole training process
    for sigma in sigmas:
        sigma_tmp=sigma
        sigma=sigma[0]*10**(-1*sigma[1])
        if Binary:
            result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        else:
            result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        results=pd.read_pickle(result_path+"_results.pkl")
        MIN=np.min(results["Bound"])
        if (MIN<optimal[1]):
            optimal[1]=MIN
            optimal[0]=sigma
    print("The optimal sigma is {} with bound value {}".format(optimal[0],optimal[1]))
    
    
    
    #### chest xray 14 
    
    
"""  
  ['No Finding' 'Pneumothorax' 'Effusion' 'Pleural_Thickening'
  'Edema' 'Atelectasis'  'Cardiomegaly' 'Consolidation' 'Pneumonia'
  'Mass' 'Infiltration' 'Fibrosis' 'Nodule' 
 'Emphysema'   'Hernia']
"""
##### chexpert
"""
    [No Finding, Pneumothorax, Pleural Effusion, 
     Edema, Atelectasis, Cardiomegaly, Consolidation, Pneumonia,
     Pleural Other, Lung Opacity,Lung Lesion,Fracture,Support Devices,Enlarged Cardiomediastinum]
    
"""
### nodule = lung lesion ?, lung opacity = infiltration? 

### paper thinks that only overlap is Cardiomegaly (CM), Edema (EDMA),
#Consolidation (CS), Atelectasis (ATL), and Pleural Effusion
#(PE) and no finding

'\n    [No Finding, Pneumothorax, Pleural Effusion, \n     Edema, Atelectasis, Cardiomegaly, Consolidation, Pneumonia,\n     Pleural Other, Lung Opacity,Lung Lesion,Fracture,Support Devices,Enlarged Cardiomediastinum]\n    \n'

# Training

In [5]:
#alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
alphas=[0,0.1]
epsilons=[0.1]#,0.01,0.001]
tasks=[4]
for t in tasks:
    x_source,y_source,x_target,y_target=load_task(t)
    for alpha in alphas:
        if alpha==0:
            pass
        else:
            print("Alpha is:"+str(alpha))
            x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source_bin,test_size=alpha,random_state=69105)
            w_a=train_prior(alpha,1,x_source,y_source,x_target=x_target,y_target=y_target,save=True,Task=t,Binary=True,batch_size=128)
        for epsilon in epsilons:
            w_s=train_posterior(alpha,x_source,y_source,None,x_test=x_source,y_test=y_source,epsilon=epsilon,Task=t,Binary=True,batch_size=128)
            #for sigma in sigmas:
                #res=read_and_prepare_results(alpha,x_source,y_source_bin,x_target,y_target_bin,sigma,delta,len(x_source),epsilon,Binary=True)#x_bound,y_bound,x_target,y_target_bin,sigma,delta,len(x_bound),epsilon,Binary=True)
                #plot_result_file(epsilon,alpha,sigma,TASK,Binary=True)

mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 0.5223306 60.093555
---------------Load USPS----------------
Training set (7438, 32, 32, 3) (7438, 10)
Test set (1860, 32, 32, 3) (1860, 10)



 !!! training error threshold reached, no further training !!!
----------------------------------------
Finished training first posterior epoch

Epoch 00001: saving model to posteriors/task4/Binary/100_0/2_1.ckpt

 !!! training error threshold reached, no further training !!!


KeyboardInterrupt: 